In [ ]:
import pandas as pd
import numpy as np

### Carga dos dados

In [ ]:
# definir arquivo Pickle de entrada
arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

df = pd.read_pickle(arquivo)
df.head()

In [34]:
df.shape

(41146, 18)

In [35]:
df.shape

(41146, 18)

### Expansão dos dados agregados

In [36]:
%%time
#df2 = df.head(10000).copy()
df2 = df.copy()
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(lambda x: [x[0]] * x[1], axis=1)
df3 = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
df3.head(200)

CPU times: user 372 ms, sys: 8.14 ms, total: 380 ms
Wall time: 379 ms


,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,165,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,165,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,165,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,165,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,165,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JOSE,1,53,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,53,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,53,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
# usar os dados expandidos invés dos agregados
df0 = df # guardar o dataframe original
df = df3 # apontar para o novo dataframe

### Decomposição com PCA

In [38]:
#from sklearn.decomposition import PCA
#X_reduced = PCA(n_components=4).fit_transform(df.drop(['SEXO'], axis=1))
#X_reduced[:5]

In [39]:
#X = pd.DataFrame(X_reduced, index=df.index)
#y = df['SEXO'] # apenas a coluna alvo
#print('Forma dos dados:', X.shape, y.shape)

### Uso dos dados originais

In [40]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (247011, 16) (247011,)


In [41]:
X.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0


In [42]:
y.head()

NOME
MARIA    0
MARIA    0
MARIA    0
MARIA    0
MARIA    0
Name: SEXO, dtype: uint8

In [43]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [44]:
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [45]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [46]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    random_state=42, multi_class='auto', 
    max_iter=1000, solver='liblinear', C=0.1) # 93.10
evaluate_model('LR', model)

params = dict(
    solver=['liblinear','lbfgs'], 
    C=np.logspace(-3,3,7)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear') 
Cross-Validation Score: 93.09 (+/- 0.06) [19759 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.7s finished


In [47]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.1) # 93.07
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(
    C=np.logspace(-4,3,8)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LinearSVC(C=0.1, random_state=42) 
Cross-Validation Score: 93.16 (+/- 0.11) [45368 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   45.3s finished


In [48]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=11) # 98.59
evaluate_model('KNN', model)

params = dict(
    n_neighbors=[1,3,5,7,9,11]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [ ]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 93.20
evaluate_model('LDA', model)

params = dict(
    solver=['svd','lsqr','eigen']
)
#fine_tune_model(model, params)

In [ ]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

In [ ]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=1.0) # 92.95
evaluate_model('GNB', model)

params = dict(
    var_smoothing=np.logspace(0,-9,num=10)
)
#fine_tune_model(model, params)

In [ ]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 52.24
evaluate_model('BNB', model)

#params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    random_state=42, criterion='gini', max_depth=17) # 97.23
evaluate_model('DT', model)

params = dict(
    criterion=['gini','entropy'],
    max_depth=[3,5,7,11,13,17,19]
)
#fine_tune_model(model, params)

In [ ]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    random_state=42, max_features='auto', n_estimators=100) # 99.80
evaluate_model('RF', model)

params = dict(
    n_estimators=[10,50,100,500], 
    max_features=['auto','sqrt','log2']
)
#fine_tune_model(model, params)

In [ ]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(
    DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), 
    n_estimators=3) # 99.76
evaluate_model('ABDT', model)

params = dict(
    n_estimators=[1,3,5,7,9,11,13]
)
#fine_tune_model(model, params)

In [ ]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=100, tol=0.01) # 91.59
evaluate_model('SGD', model)

params = dict(
    max_iter=[100, 200, 350, 500, 1000], 
    tol=[0.01, 0.1, 1.0]
)
#fine_tune_model(model, params)

In [ ]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=750, tol=0.1) # 90.87
evaluate_model('PCT', model)

params = dict(
    max_iter=[100, 200, 350, 500, 750, 1000], 
    tol=[0.1, 0.01, 0.001]
)
#fine_tune_model(model, params)

In [ ]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org xgboost

model = XGBClassifier(
    use_label_encoder=False, eval_metric='logloss',
    max_depth=9, min_child_weight=1, gamma=0.4, 
    subsample=0.9, colsample_bytree=0.6, reg_alpha=1e-05) # 99.20
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 99.85
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 99.85
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 99.85
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 99.85
#fine_tune_model(model, params)

In [ ]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(
    num_leaves=97, max_depth=11, 
    learning_rate=0.1, feature_fraction=0.5) # 97.09
evaluate_model('LGBM', model)

#params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 99.85
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 99.85
params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 99.87
#fine_tune_model(model, params)

In [ ]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier(
    silent=True, iterations=500,
    learning_rate=0.05, depth=9) # 99.61
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(
    iterations=[10,50,100,250,500,1000],
    learning_rate=[0.01,0.05,0.1,0.5],
    depth=range(1,11,2)
) 
#fine_tune_model(model, params)

In [ ]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
    (model, score, stdev, elapsed) = value
    names.append(key)
    estimators.append(model)
    scores.append(score)
    stdevs.append(stdev)
    times.append(elapsed)

In [ ]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

In [32]:
model = models['LGBM'][0]
model

LGBMClassifier(feature_fraction=0.5, max_depth=11, num_leaves=97)

In [33]:
# treinar o melhor modelo com todos os dados
model.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5


LGBMClassifier(feature_fraction=0.5, max_depth=11, num_leaves=97)

## Avaliação do melhor modelo

In [34]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df2 = pd.read_pickle(arquivo)
df2.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,165,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,53,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,233,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,91,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,202,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [35]:
df2.shape

(41146, 18)

In [36]:
X_test = df2.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [37]:
y_pred = model.predict(X_test)
y_pred

array([0, 1, 0, ..., 0, 1, 1], dtype=uint8)

In [38]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Accuracy score: 99.42 %


In [39]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Confusion matrix:
 [[21766   124]
 [  115 19141]]


In [40]:
dados = pd.DataFrame({'REAL': y_test, 'PREV': y_pred}, index=X_test.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'F', 1: 'M', 9: 'X'})
dados.head()

,REAL,PREV
NOME,,
MARIA,F,F
JOSE,M,M
ANA,F,F
JOAO,M,M
CARLOS,M,M


In [41]:
dados[dados['REAL'] != dados['PREV']].head(20)

,REAL,PREV
NOME,,
GEOVANI,F,M
BRUCE,M,F
SALIM,M,F
MUNIQUE,M,F
FREDERIC,M,F
MARINEY,M,F
PATRIC,M,F
LIRIAN,M,F
ALI,F,M
